In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import cv2 

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout,GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard,CSVLogger
img_width=2048 # 2048
img_height=2048 # 2048

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
defect_folder= '/content/drive/MyDrive/dataset_train/spay_defects_half'


In [4]:
normal_folder= '/content/drive/MyDrive/dataset_train/normal_half'


In [5]:
import tensorflow as tf
from tensorflow import keras

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

In [6]:
def normalize(img):
    min = img.min()
    max = img.max()
    return 2.0 * (img - min) / (max - min) - 1.0

In [7]:
def norm_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if not filename.startswith('.'):
            img=cv2.imread(os.path.join(folder,filename))[:2048,:2048,:3]#2048
      #      img = cv2.fastNlMeansDenoising(img,50,50,20,21) 
            img = normalize(img)
            if img is not None:
                images.append(img)
    return images

In [8]:
img_defects_norm=norm_images_from_folder(defect_folder)
#crashes here on 12GB ram. had to reduce data and increase ram.

In [9]:
img_clean_norm=norm_images_from_folder(normal_folder)


In [10]:
img_all=img_defects_norm+img_clean_norm
Y=[1]*len(img_defects_norm)+[0]*len(img_clean_norm)
np.save("img_all.npy",img_all)
np.save("Y_all.npy",Y)


In [11]:
img_all=np.load("img_all.npy")
Y=np.load("Y_all.npy")

In [12]:
img_clean_norm = None
img_defects_norm = None

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(img_all, Y, 
                    test_size=0.2, stratify = Y, random_state=0)

In [14]:
y_train=np.array(y_train)



In [15]:
y_test=np.array(y_test)


In [16]:
X_train=np.array(X_train)
#crashes here(need more RAM). saving data as .npy recovered 10gb ram

In [17]:
X_test=np.array(X_test)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
print(y_train[:10])

(72, 2048, 2048, 3) (72,) (19, 2048, 2048, 3) (19,)
[0 1 1 0 1 0 1 1 0 0]


In [18]:
model = tf.keras.applications.InceptionV3(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))#

# Freeze the layers which you don't want to train. Google how to freeze first 20/25 layers 
for layer in model.layers[0:305]:#total 311 layers for Inception3, so just keeping last 5 layer flexible
    layer.trainable = False
for layer in model.layers:
    print(layer, layer.trainable)


87916544/87910968 [==============================] - 0s 0us/step
<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f18c0466490> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f18bea54c90> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f18bea1f850> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f18be9d8c90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f18aa964b50> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f18aa982e50> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f18aa92c350> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f18aa936b90> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f18aa93b910> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f18aa9362d0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D 

In [19]:
#Adding custom Layers
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(5,activation='softmax')(x)
predictions = Dense(1, activation="sigmoid")(x)
# creating the final model 
model_final = Model(model.input, predictions)
model_final.compile(loss='binary_crossentropy',
                   optimizer='adam',
                    metrics=['accuracy'])#change for F1score
                   # options = run_opts,
                   # run_metadata=runmeta)

print(model_final.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2048, 2048,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1023, 1023, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1023, 1023, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 1023, 1023, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [20]:
model_path = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
early_stopping = EarlyStopping(patience=5)
model_checkpoint = ModelCheckpoint(model_path,
                                   save_best_only=True, save_weights_only=True)

csv_logger = CSVLogger("InceptionV3.csv", append=True)
#name='vgg19_denoise'
tensorboard = TensorBoard(log_dir='InceptionV3_log')

def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 :
    return 1e-4

model_final.fit(X_train, y_train,validation_data=(X_test, y_test),
                     epochs=15,batch_size=3, shuffle=True,
                     verbose=2,
                     callbacks=[model_checkpoint, early_stopping,csv_logger,
                                tf.keras.callbacks.LearningRateScheduler(decay),tensorboard])


Epoch 1/15
24/24 - 87s - loss: 0.6942 - accuracy: 0.5278 - val_loss: 0.6780 - val_accuracy: 0.5789
Epoch 2/15
24/24 - 79s - loss: 0.6869 - accuracy: 0.5556 - val_loss: 0.6675 - val_accuracy: 0.5789
Epoch 3/15
24/24 - 79s - loss: 0.6804 - accuracy: 0.5556 - val_loss: 0.6621 - val_accuracy: 0.5789
Epoch 4/15
24/24 - 77s - loss: 0.6754 - accuracy: 0.5556 - val_loss: 0.6615 - val_accuracy: 0.5789
Epoch 5/15
24/24 - 77s - loss: 0.6793 - accuracy: 0.5556 - val_loss: 0.6623 - val_accuracy: 0.5789
Epoch 6/15
24/24 - 76s - loss: 0.6745 - accuracy: 0.5556 - val_loss: 0.6627 - val_accuracy: 0.5789
Epoch 7/15
24/24 - 76s - loss: 0.6718 - accuracy: 0.5556 - val_loss: 0.6610 - val_accuracy: 0.5789
Epoch 8/15
24/24 - 76s - loss: 0.6725 - accuracy: 0.5556 - val_loss: 0.6604 - val_accuracy: 0.5789
Epoch 9/15
24/24 - 76s - loss: 0.6745 - accuracy: 0.5556 - val_loss: 0.6602 - val_accuracy: 0.5789
Epoch 10/15
24/24 - 76s - loss: 0.6732 - accuracy: 0.5556 - val_loss: 0.6601 - val_accuracy: 0.5789
Epoch 11/

In [21]:
# report
pred=model_final.predict(X_test)
pred_Y=np.argmax(pred,axis=1)
print("pred:", pred_Y)
print("actual:", y_test)

pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
actual: [0 0 1 1 0 0 0 1 0 1 1 1 0 1 0 1 1 1 1]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
#plt.gca().set_ylim(0,1)
plt.show()
#sgd

In [22]:
model = tf.keras.applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

# Freeze the layers which you don't want to train. Google how to freeze first 20/25 layers 
for layer in model.layers[20]:
    layer.trainable = False
#Adding custom Layers 
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)#changed from 512, 256
x = Dense(512, activation="relu")(x)#changed from 512
x=Dropout(0.2)(x)
x = Dense(128, activation="relu")(x)
x = Dense(56, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)
# creating the final model 
model_final = Model(model.input, predictions)
model_final.compile(loss='binary_crossentropy',
                   optimizer='adam',
                    metrics=['accuracy'])#change for F1score
                   # options = run_opts,
                   # run_metadata=runmeta)

print(model_final.summary())

94773248/94765736 [==============================] - 0s 0us/step


TypeError: ignored

In [ ]:


model_path = 'ResNet50.h5'
early_stopping = EarlyStopping(patience=5)
model_checkpoint = ModelCheckpoint(model_path,
                                   save_best_only=True, save_weights_only=True)

csv_logger = CSVLogger("ResNet50.csv", append=True)
#name='vgg19_denoise'
tensorboard = TensorBoard(log_dir='ResNet50_log')

def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 :
    return 1e-4

model_final.fit(X_train, y_train,validation_data=(X_test, y_test),
                     epochs=15,batch_size=3, shuffle=True,
                     verbose=2,
                     callbacks=[model_checkpoint, early_stopping,csv_logger,
                                tf.keras.callbacks.LearningRateScheduler(decay),tensorboard])



In [ ]:
# report
pred=model_final.predict(X_test)
pred_Y=np.argmax(pred,axis=1)
print("pred:", pred_Y)
print("actual:", y_test)